In [120]:
import csv
from pyspark.sql.functions import min, max
from pyspark.sql.functions import col
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

# load the data
movies_file = sc.textFile("dataset/movies.csv")
genres_file = sc.textFile("dataset/genres.csv")
actors_file = sc.textFile("dataset/actors.csv")
tagNames_file = sc.textFile("dataset/tag_names.csv")
tags_file = sc.textFile("dataset/tags.csv")

# we separate the fields for each table in csv format
data_movies = movies_file.map(lambda row: next(csv.reader(row.splitlines(), skipinitialspace=True)))
data_genres = genres_file.map(lambda row: next(csv.reader(row.splitlines(), skipinitialspace=True)))
data_actors = actors_file.map(lambda row: next(csv.reader(row.splitlines(), skipinitialspace=True)))
data_tagNames = tagNames_file.map(lambda row: next(csv.reader(row.splitlines(), skipinitialspace=True)))
data_tags = tags_file.map(lambda row: next(csv.reader(row.splitlines(), skipinitialspace=True)))

# we create the dataframe for each data generated
table_movies = spark.createDataFrame(data_movies, ['mid', 'title','year','rating','num_ratings'])
table_genres = spark.createDataFrame(data_genres, ['mid', 'genre'])
table_actors = spark.createDataFrame(data_actors, ['mid', 'name', 'cast_position'])
table_tagNames = spark.createDataFrame(data_tagNames, ['tid', 'tag'])
table_tags = spark.createDataFrame(data_tags, ['mid', 'tid'])

# create an alias for each table
movies = table_movies.alias('movies')
genres = table_genres.alias('genres')
actors = table_actors.alias('actors')
tagNames = table_tagNames.alias('tagNames')
tags = table_tags.alias('tags')

# we mantain the tables in cache

genres.persist()
actors.persist()
tagNames.persist()
tags.persist()
movies.persist()

mid,title,year,rating,num_ratings
1,Toy story,1995,3.7,102338
2,Jumanji,1995,3.2,44587
3,Grumpy Old Men,1993,3.2,10489
4,Waiting to Exhale,1995,3.3,5666
5,Father of the Bri...,1995,3,13761
6,Heat,1995,3.9,42785
7,Sabrina,1954,3.8,12812
8,Tom and Huck,1995,2.7,2649
9,Sudden Death,1995,2.6,3626
10,GoldenEye,1995,3.4,28260


### 1. Print all movie titles starring ‘Daniel Craig’, sorted in an ascending alphabetical order.

In [121]:
movies_with_actors = movies.join(actors, movies.mid == actors.mid)

movies_title_actors = movies_with_actors.select('title','name')
movies_with_Craig = movies_title_actors.filter(movies_title_actors.name == 'Daniel Craig').sort(movies_title_actors.title.asc())

# display the result: we use count to display all the rows
#movies_with_Craig.show(movies_with_Craig.count(), truncate=False)
movies_with_Craig

title,name
A Kid in King Art...,Daniel Craig
Archangel,Daniel Craig
Casino Royale,Daniel Craig
Casino Royale,Daniel Craig
Elizabeth,Daniel Craig
Enduring Love,Daniel Craig
Infamous,Daniel Craig
Lara Croft: Tomb ...,Daniel Craig
Layer Cake,Daniel Craig
Munich,Daniel Craig


### 2. Print names of the cast of the movie ‘The Dark Knight’ in an ascending alphabetical order.

In [110]:
movies_with_actors = movies.join(actors, movies.mid == actors.mid)
movies_title_actors = movies_with_actors.select('title','name')

cast_TheDarkKnight = movies_title_actors.filter(movies_title_actors.title == 'The Dark Knight').sort(movies_title_actors.name.asc())
only_cast = cast_TheDarkKnight.select('name')
#only_cast.show(only_cast.count(), truncate=False)
only_cast

name
Aaron Eckhart
Adam Kalesperis
Aidan Feore
Andrew Bicknell
Andy Luther
Anthony Michael Hall
Ariyon Bakare
Beatrice Rosen
Bill Smille
Brandon Lambdin


### 3. Print the distinct genres in the database and their corresponding number of movies N where N is greater than 1000, sorted in the ascending order of N

In [28]:
reduced_genres = genres.rdd.map(lambda x: (x.genre,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: (int(x[1]) > 1000))
#reduced_genres.takeOrdered(20, key = lambda x: -x[1])
reduced_genresdf = reduced_genres.toDF(['genre', 'countMovies']).sort('countMovies')
reduced_genresdf

genre,countMovies
Adventure,1003
Crime,1086
Action,1445
Romance,1644
Thriller,1664
Comedy,3566
Drama,5076


### 4. For each year, print the movie title, year, and rating, sorted in the ascending order of year and the descending order of movie rating

In [29]:
movies_sorted= movies.sort(movies.year.asc(), movies.rating.desc())
movies_sortedFiltered = movies_sorted.select('title', 'year', 'rating')
movies_sortedFiltered

title,year,rating
The Great Train R...,1903,0
The Birth of a Na...,1915,3.3
Intolerance: Love...,1916,3.8
The Immigrant,1917,0
Otets Sergiy,1917,0
A Dog's Life,1918,0
Broken Blossoms o...,1919,3.7
"Die Spinnen, 1. T...",1919,0
Male and Female,1919,0
Das Cabinet des D...,1920,4.1


### 5. Critiques say that some words used in tags to convey emotions are very recurrent. To convey positive and negative emotions, the words ‘good’ and ‘bad’, respectively, are used predominantly in tags. Print all movie titles whose audience opinion is split (i.e., has at least one audience who expresses positive emotion and at least one who expresses negative emotion).

In [76]:
movies_with_tags = movies.join(tags, ['mid']).join(tagNames, ['tid'])
movies_with_tags_onlyGood = movies_with_tags.filter(movies_with_tags.tag.contains( 'good'))
movies_with_tags_onlyBad = movies_with_tags.filter(movies_with_tags.tag.contains( 'bad'))

movies_with_tags_onlyGoodBad = (movies_with_tags_onlyGood.select('mid','title')).intersect(movies_with_tags_onlyBad.select('mid','title'))
result = movies_with_tags_onlyGoodBad.select('title')
result


title
Twilight
The Forgotten
Starship Troopers
Bridget Jones's D...
Howard the Duck
Hercules in New York
C.H.U.D.
The Wicker Man
Ocean's Eleven
Return of the Kil...


### 6. One would expect that the movie with the highest number of user ratings is either the highest rated movie or perhaps the lowest rated movie. Let’s find out if this is the case here:

#### 6.1 : Print all information (mid, title, year, num ratings, rating) for the movie(s) with the highest number of ratings

In [163]:
num_ratings = movies.filter(movies.num_ratings != '\\N').withColumn('num_ratings', col('num_ratings').cast('int'))
max_num_rating = num_ratings.select(max('num_ratings'))

movies_with_max_num_ratings = movies.filter(movies.num_ratings == max_num_rating.collect()[0][0])
movies_with_max_num_ratings


mid,title,year,rating,num_ratings
4201,Pirates of the Ca...,2007,3.8,1768593
53125,Pirates of the Ca...,2007,3.8,1768593


#### 6.2: Print all information (mid, title, year, num ratings, rating) for the movie(s) with the highest rating (include tuples that tie), sorted by the ascending order of movie id.

In [162]:
num_ratings = movies.filter(movies.rating != '\\N').withColumn('rating', col('rating').cast('double'))
max_rating = num_ratings.select(max('rating'))
movies_with_max_ratings = movies.filter(movies.rating == max_rating.collect()[0][0])
movies_with_max_ratings


mid,title,year,rating,num_ratings
4311,1732 Høtten,1998,5,5


#### 6.3: Is (Are) the movie(s) with the most number of user ratings among these highest rated movies? Print the output of the query that will check our conjecture (i.e., your query will print the movie(s) that has (have) the highest number of ratings as well as the highest rating)

In [164]:
result = movies_with_max_num_ratings.intersect(movies_with_max_ratings)
result

mid,title,year,rating,num_ratings


#### 6.4: Print all information (mid, title, year, num ratings, rating) for the movie(s) with the lowest rating (include tuples that tie), sorted by the ascending order of movie id.

In [172]:
num_ratings = movies.filter(movies.rating != '\\N').withColumn('rating', col('rating').cast('double'))
min_rating = num_ratings.select(min('rating'))
movies_with_min_ratings = movies.filter(movies.rating == min_rating.collect()[0][0])
movies_with_min_ratings




mid,title,year,rating,num_ratings
32,Twelve Monkeys,1995,0,0
33,Wings of Courage,1995,0,0
39,Clueless,1995,0,0
56,Kids of the Round...,1997,0,0
59,Le confessionnal,1995,0,0
61,Eye for an Eye,1996,0,0
63,Don't Be a Menace...,1996,0,0
69,Friday,1995,0,0
115,Le bonheur est da...,1995,0,0
126,The Neverending S...,1994,0,0


#### 6.5: Is (Are) the movie(s) with the most number of user ratings among these lowest rated movies? Print the output of the query that will check our conjecture (i.e., your query will print the movie(s) that has (have) the highest number of ratings as well as the lowest rating)

In [173]:
result = movies_with_max_num_ratings.intersect(movies_with_min_ratings)
result

mid,title,year,rating,num_ratings


#### 6.6: In conclusion, is our hypothesis or conjecture true for the MovieLens database?

No, the results of query 6.3 and 6.5 show that both the intersection of max_num_ratings and max_rating/min_rating are empty! 